<a href="https://colab.research.google.com/github/Traivok/tcc/blob/develop/scripts/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi
! pip install ultralytics --upgrade
! pip install wandb
! pip install roboflow
! pip install scikit-learn
! pip install python-dotenv

/bin/bash: line 1: nvidia-smi: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.0/887.0 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.3.31
    Uninstalling ultralytics-8.3.31:
      Successfully uninstalled ultralytics-8.3.31


In [6]:
import ultralytics
import warnings
import torch
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [21]:
%matplotlib inline
ultralytics.checks()

Ultralytics 8.3.31 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.9/107.7 GB disk)


In [16]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

from dotenv import load_dotenv
load_dotenv()

ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

if IN_COLAB:
  from google.colab import userdata
  ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')



In [17]:
# Check if running on a MacBook with M1 chip
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) for hardware acceleration on MacBook M1.")
# Check if CUDA is available
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA for hardware acceleration.")
else:
    device = torch.device("cpu")
    warnings.warn("Using CPU for computation. This may be slower than using MPS or CUDA.", UserWarning)

<ipython-input-17-b61d74a175b2>:11: UserWarning: Using CPU for computation. This may be slower than using MPS or CUDA.
  warnings.warn("Using CPU for computation. This may be slower than using MPS or CUDA.", UserWarning)


In [18]:
base_path = os.getcwd()
print(f"Current working directory: {base_path}")

# Example usage with Ultralytics
model_list =  [
    #  'yolov3u.pt',
    #  'yolov5mu.pt',
     'yolov8m.pt',
    #  'yolov9m.pt',
    #  'yolov10m.pt',
    #  'yolo11m.pt',
]

for model_name in model_list:
        print(model_name)
        model = ultralytics.YOLO(model_name).to(device)

Current working directory: /content
yolov3u.pt


100%|██████████| 198M/198M [00:04<00:00, 45.8MB/s]


In [19]:
from roboflow import Roboflow

print(f"Roboflow API key: {ROBOFLOW_API_KEY}")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("ze").project("tcc-bbcjn")
version = project.version(1)
dataset = version.download("yolov8")

Roboflow API key: SpzA9wCLPkImtTIJ3srw
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tcc-1 in yolov8:: 100%|██████████| 10434/10434 [00:02<00:00, 3719.32it/s]


In [ ]:
from google.colab import drive
import shutil

# Monta o Google Drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO

DATASET = dataset.location + "/data.yaml"
MODEL_NAME=model_list[0] ## should be yolov8
PATIENCE=30
EPOCHS=100
IMGSZ=(640, 640)
RUN_NAME=f'tcc_{MODEL_NAME}'

model = YOLO(MODEL_NAME)
results = model.train(
    task='detect',
    data=DATASET,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=32,
    patience=10,
    project='tcc',
    name=RUN_NAME,
    save=True,
    save_period=5,
    cache=True,
    plots=True,
    seed=0)

In [ ]:
# Define o caminho de destino no Google Drive
destination_dir = '/content/drive/My Drive/tcc'
destination_path = os.path.join(destination_dir, 'best.pt')


# Verifica se o diretório existe; se não, cria-o
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Copia o arquivo para o destino no Google Drive
shutil.copy(f'/content/runs/{RUN_NAME}/weights/best.pt', destination_path)

print("Arquivo enviado para o Google Drive com sucesso!")